In [2]:
import pandas as pd 
from matplotlib import pyplot as plt 
import numpy as np 
import matplotlib as mpl
% matplotlib inline 
import plotly.plotly as ply
import cufflinks as cf
import plotly.graph_objs as go
from datetime import datetime
import plotly

#I also imported my credentials for ploty here

In [3]:
ls

eletr_gas2.csv           energy_timeseries.ipynb


In [6]:
data = pd.read_csv('eletr_gas2.csv', sep=';') # index_col=0
data.head()


,DATE,IPG2211A2N
0,01/01/1985,72.6803
1,01/02/1985,70.8479
2,01/03/1985,62.6166
3,01/04/1985,57.6106
4,01/05/1985,55.4467


In [7]:
data.dtypes

DATE           object
IPG2211A2N    float64
dtype: object

In [40]:
list(data)

['IPG2211A2N']

In [27]:
data['DATE'] = data['DATE'].apply(lambda x: datetime.strptime(x, '%d/%m/%Y'))

TypeError: strptime() argument 1 must be string, not Timestamp

In [30]:
data.dtypes

DATE          datetime64[ns]
IPG2211A2N           float64
dtype: object

In [38]:
data=data.set_index(['DATE'])

In [39]:
data.dtypes

IPG2211A2N    float64
dtype: object

In [40]:
data.columns = ['Energy Consumption']

In [7]:
list(data)

['Energy Consumption']

In [43]:
dataplot = [go.Scatter(x= data.index, y=data['Energy Consumption'])]
ply.iplot(dataplot, filename='pandas-time-series')

In [44]:
from pyramid.arima import auto_arima
stepwise_model = auto_arima(data, start_p=1, start_q=1,
                           max_p=3, max_q=3, m=12,
                           start_P=0, seasonal=True,
                           d=1, D=1, trace=True,
                           error_action='ignore',  
                           suppress_warnings=True, 
                           stepwise=True,
                           max_order=4)


/Users/Susana/anaconda2/lib/python2.7/site-packages/statsmodels/compat/pandas.py:56: FutureWarning:

The pandas.core.datetools module is deprecated and will be removed in a future version. Please use the pandas.tseries module instead.



Fit ARIMA: order=(1, 1, 1) seasonal_order=(0, 1, 1, 12); AIC=1775.702, BIC=1795.609, Fit time=4.895 seconds
Fit ARIMA: order=(0, 1, 0) seasonal_order=(0, 1, 0, 12); AIC=nan, BIC=nan, Fit time=nan seconds
Fit ARIMA: order=(1, 1, 0) seasonal_order=(1, 1, 0, 12); AIC=1941.061, BIC=1956.987, Fit time=1.543 seconds
Fit ARIMA: order=(0, 1, 1) seasonal_order=(0, 1, 1, 12); AIC=1833.016, BIC=1848.942, Fit time=1.543 seconds
Fit ARIMA: order=(1, 1, 1) seasonal_order=(1, 1, 1, 12); AIC=1776.222, BIC=1800.111, Fit time=6.046 seconds
Fit ARIMA: order=(1, 1, 1) seasonal_order=(0, 1, 0, 12); AIC=1905.777, BIC=1921.703, Fit time=2.916 seconds
Fit ARIMA: order=(1, 1, 1) seasonal_order=(0, 1, 2, 12); AIC=1774.910, BIC=1798.798, Fit time=49.100 seconds
Fit ARIMA: order=(0, 1, 1) seasonal_order=(0, 1, 2, 12); AIC=1832.054, BIC=1851.961, Fit time=19.577 seconds
Fit ARIMA: order=(1, 1, 0) seasonal_order=(0, 1, 2, 12); AIC=1852.434, BIC=1872.341, Fit time=12.518 seconds
Fit ARIMA: order=(0, 1, 0) seasonal_o

In [45]:
print(stepwise_model.aic()) 

1774.90984971


In [46]:
train_set = data.loc['1985-01-01':'2015-12-01']
test_set = data.loc['2016-01-01':]

In [47]:
test_set.head()

,Energy Consumption
DATE,
2016-01-01,117.2128
2016-02-01,106.2540
2016-03-01,94.4715
2016-04-01,88.3776
2016-05-01,89.6237


In [48]:
test_set.tail()

,Energy Consumption
DATE,
2017-08-01,108.0286
2017-09-01,97.2464
2017-10-01,92.0177
2017-11-01,96.5025
2017-12-01,114.6551


In [49]:
test_set.iplot()

In [50]:
stepwise_model.fit(train_set)

ARIMA(callback=None, disp=0, maxiter=50, method=None, order=(1, 1, 1),
   out_of_sample_size=0, scoring='mse', scoring_args={},
   seasonal_order=(0, 1, 2, 12), solver='lbfgs', start_params=None,
   suppress_warnings=True, transparams=True, trend='c')

In [51]:
forecast = stepwise_model.predict(n_periods=24)

In [52]:
print(forecast)

[ 114.74357914  109.3000312   100.97819419   89.22595148   91.18428909
  102.28612134  111.43035645  110.58521997  100.46680721   91.53287372
   94.5368581   107.25387837  117.10637366  109.89632507  101.40948477
   89.6758271    91.47927068  102.24920171  111.27597068  110.4596246
  100.16973673   91.52461004   94.87288549  107.90952805]


In [53]:
future_forecast=pd.DataFrame(forecast, index=test_set.index, columns=['Prediction'])

In [54]:
pd.concat([test_set,future_forecast]).iplot()